In [1]:
# 05-04 update
# This version of the program adds some basic analyzing functions based on
# networkx V2.

# 04-17 update
# This version of the program is a beta version for pickle to networkx V1,
# it aims to handel multi graphs situations in QN, connect the main graph and sub graph,
# In networkx, the sub graph is represented by a single node;
# In pyvis, the sub graph is represented by an individual html, linked with the representing
# node in the future update. 

# 04-24 update
# This version aims to deal with the same nodes of different sub-graph, an overall net would 
# be built for the calculation and evaluation in the future. 

# part 1 
# basic definition 

import networkx as nx 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json 
import pickle
import os

from pyvis.network import Network
from PyQt5.QtCore import QDateTime, Qt, QTimer
from PyQt5.QtWidgets import (QApplication, QCheckBox, QComboBox, QDateTimeEdit,
        QDial, QDialog, QGridLayout, QGroupBox, QHBoxLayout, QLabel, QLineEdit,
        QProgressBar, QPushButton, QRadioButton, QScrollBar, QSizePolicy,
        QSlider, QSpinBox, QStyleFactory, QTableWidget, QTabWidget, QTextEdit,
        QVBoxLayout, QWidget, QToolTip, QTableWidget, QTableWidgetItem)
from PyQt5.QtGui import QIcon, QPixmap
from PyQt5.QtGui import QFont

In [2]:
# part 2
# import the data pickle version

# this will decide the color map
nwtype = 'empirical_study'#'definition'

# import from the database
# f = open('database/curiosity.p','rb')
fdir = '../database/3d printing/pickle/'

file_list = os.listdir(fdir)
file_index = len(file_list)

# initilize the variable for graph
prepare_list = locals()

# build the individual main graph and sub graph
# index 0: for the main graph
# index from 1: for the sub graph, if exists
for i in range(file_index):
    pickfile=open(fdir+'%s'%file_list[i],'rb')
    mydict = pickle.load(pickfile)
    pickfile.close()
    
    # n_str(i) used for the variable for the networkx construction
    # p_str(i) used for the variable for the pyvis display 
    prepare_list['n_'+str(i)] = nx.Graph()
    prepare_list['p_'+str(i)] = Network(height="100%", width="100%", bgcolor="#222222", font_color="white")
    
    # divde the node and link 
    [dic_node, dic_link] = mydict
    
    # set the color map 
    color_map = []
    rgb_map = []
    title_name = []

    # add nodes into the graph
    (row, col) = dic_node.shape
    for index in range(row):
        nd_id = dic_node.iloc[index]['id']
        nd_content = dic_node.iloc[index]['content']
        nd_feature = dic_node.iloc[index]['feature']
        nd_label = dic_node.iloc[index]['label']
        nd_reference = dic_node.iloc[index]['reference']
        prepare_list['n_'+str(i)].add_node(nd_id,feature=nd_feature,content=nd_content,label=nd_label,reference=nd_reference)
        
        # the default node size
        node_size = 12
        # build the color map 
        if nwtype == 'definition':
            if nd_feature == 'Central question':
                rgb_map.append({'r': 205, 'g': 37, 'b': 38, 'a': 0})
                color_map.append('red')
                node_size = 20
                title_name = nd_label
            elif nd_feature == 'Question':
                rgb_map.append({'r': 205, 'g': 137, 'b': 38, 'a': 0})
                color_map.append('orange')
                node_size = 15
            else:
                rgb_map.append({'r': 144, 'g': 202, 'b': 249, 'a': 0})
                color_map.append('blue')
            
        elif nwtype == 'empirical_study':        
            if nd_feature == 'Central question':
                rgb_map.append({'r': 205, 'g': 37, 'b': 38, 'a': 0})
                color_map.append('red')
                node_size = 20
                title_name = nd_label
            elif nd_feature == 'question':
                rgb_map.append({'r': 205, 'g': 137, 'b': 38, 'a': 0})
                color_map.append('orange')
                node_size = 15
            elif nd_feature == 'answer':
                rgb_map.append({'r': 144, 'g': 202, 'b': 249, 'a': 0})
                color_map.append('blue')
            elif nd_feature == 'Statement':
                rgb_map.append({'r': 199, 'g': 129, 'b': 32, 'a': 0})
                color_map.append('green')
    
        node_color = color_map[-1]
        #     print(color)    
        prepare_list['p_'+str(i)].add_node(nd_label,size = node_size, color = node_color, feature=nd_feature,content=nd_content,reference=nd_reference)
#         print(color_map)
    
    # add links into the graph
    (row, col) = dic_link.shape
    for index in range(row):
        lk_id = dic_link.iloc[index]['id']
        lk_from = dic_link.iloc[index]['start']
        lk_to = dic_link.iloc[index]['end']
        lk_label = dic_link.iloc[index]['label']
        prepare_list['n_'+str(i)].add_edge(lk_from,lk_to,label=lk_label,id_lk=lk_id)
    
    # #     index_from = dic_node.index[dic_node['id']==lk_from].tolist()
    #     index_from = dic_link[dic_link['id']==lk_from].index.values.astype(int)
    # #     index_from = int(index_from)
    #     lk_from_n = dic_node.iloc[index_from]['label']
    # #     index_to = dic_node.index[dic_node['id']==lk_to].tolist()
    #     index_to = dic_link[dic_link['id']==lk_to].index.values
    # #     index_to = int(index_to)
    #     lk_to_n = dic_node.iloc[index_to]['label']
        lk_from_n = dic_node.loc[dic_node['id']==lk_from,'label'].values[0]
        lk_to_n = dic_node.loc[dic_node['id']==lk_to,'label'].values[0]
    
        prepare_list['p_'+str(i)].add_edge(lk_from_n,lk_to_n,label=lk_label)

# connect the main graph and subgraph
# version 1
# add the entire sub graph as a node in the main graph, therefore, the 
# complete QN contains multi graphs.


#     # connect the main graph and subgraph
#     # networkx part 
#     if i>0: # sub graph exists
#         print(i)
#         # name = prepare_list['n_'+str(1)].nodes[0]['id']
#         name = []
#         for index_id in prepare_list['n_'+str(i)].nodes():
#             if prepare_list['n_'+str(i)].nodes[index_id]['feature'] == 'Central question':
#                 name = prepare_list['n_'+str(i)].node[index_id]['label']

#         # delete the original node with same label
#         # find the index of the related node, edge 
#         id_node = []

#         id_edge_start = []
#         label_edge_start = []
#         id_lk_edge_start = []

#         id_edge_end = []
#         label_edge_end = []
#         id_lk_edge_end = []

#         for index_id in prepare_list['n_'+str(0)].nodes():
#             if len(prepare_list['n_'+str(0)].nodes[index_id])>1 and prepare_list['n_'+str(0)].nodes[index_id]['label'] == name:
#                 index = index_id
#                 id_node.append(index)
        
#                 # delete the related edge and add new edge
#                 for index_id in prepare_list['n_'+str(0)].edges():
#                     if index_id[0] == index:
#                         id_edge_end.append(index_id[1])
#                         label_edge_end.append(prepare_list['n_'+str(0)].edges[index_id]['label'])
#                         id_lk_edge_end.append(prepare_list['n_'+str(0)].edges[index_id]['id_lk'])
#                     if index_id[1] == index:
#                         id_edge_start.append(index_id[0])
#                         label_edge_start.append(prepare_list['n_'+str(0)].edges[index_id]['label'])
#                         id_lk_edge_start.append(prepare_list['n_'+str(0)].edges[index_id]['id_lk'])
                        
        
#         # delete the related node
#         if len(id_node) == 0:
#             print('Please check the original file to make sure the consistency between sub graph and representing node')
#         prepare_list['n_'+str(0)].remove_node(id_node[0])

#         # add the sub graph as a new node
#         prepare_list['n_'+str(0)].add_node(prepare_list['n_'+str(i)],label=name,feature = 'sub graph')

#         # add the original edges related with sub graph
#         len_edge_start = len(id_edge_start)
#         for index in range(len_edge_start):
#             prepare_list['n_'+str(0)].add_edge(id_edge_start[index],prepare_list['n_'+str(i)], label=label_edge_start[index],id_lk=id_edge_start[index])
#             print(prepare_list['n_'+str(0)].edges(id_edge_start[index],prepare_list['n_'+str(i)]))

#         len_edge_end = len(id_edge_end)
#         for index in range(len_edge_end):
#             prepare_list['n_'+str(0)].add_edge(prepare_list['n_'+str(i)],id_edge_end[index],label=label_edge_end[index],id_lk=id_edge_end[index])
#             print(prepare_list['n_'+str(0)].edges(prepare_list['n_'+str(i)],id_edge_end[index]))
        
#         # revise the main graph and connect the sub graph, pyvis part
#         node_list = prepare_list['p_'+str(0)].get_network_data()[0]
#         for node_p in node_list:
#             if node_p['label'] == name:
#                 node_p['color'] = 'yellow' # change the sub graph representing node to yellow


# connect the main graph and subgraph
# version 2
# add the entire sub graph directly the main graph, therefore, the 
# complete QN contains single graphs.
    
#     build the complete graph, combine the same node in different sub graph
#     initialization 
    if i == 0:
        QN = prepare_list['n_'+str(i)]
        # N = prepare_list['p_'+str(i)]
    
    # add all nodes from the sub graph 
    if i>0:
        QN.add_nodes_from(prepare_list['n_'+str(i)].nodes(data=True))
        QN.add_edges_from(prepare_list['n_'+str(i)].edges(data=True))

# connect the main graph and subgraph
# combine the nodes and attributes in Networkx
c_node = []
c_id = []
c_len = len(list(QN.nodes.data()))
c_id_full = []

# store the id of the whole network 
for index_id in QN.nodes():
    c_id_full.append(index_id)

for index in range(c_len):
    index_id = c_id_full[index]
    node_label = QN.nodes[index_id]['label']
    node_id = []
    node_id.append(index_id)
        
    flag = 0
    for index1 in range(index+1,c_len):
        index_id1 = c_id_full[index1]
        if QN.nodes[index_id1]['label'] == node_label:
            flag = flag + 1
#             print(flag)
            node_id.append(index_id1)
#             print(node_id)
            
#         print(node_id)
    
    
#     print(node_id)
    if flag > 0:
#         print(node_id)
        c_node.append(node_label)
        c_id.append(node_id)
        
print(len(c_node))
print(len(c_id))
for index in range(len(c_node)):
    print(c_node[index])
    print(c_id[index])
# print(c_node)
# print(c_id)

# combine the node with same label
for index1 in range(len(c_id)):
    for index2 in range(len(c_id[index1])-1):
        node_1 = c_id[index1][0]
        node_2 = c_id[index1][index2+1]
        QN = nx.contracted_nodes(QN, node_1, node_2)

# build the html version of the network
QN_html = Network(height="100%", width="100%", bgcolor="#222222", font_color="white")
color_map = []
for index_id in QN.nodes():
    nd_label = QN.nodes[index_id]['label']
    nd_feature = QN.nodes[index_id]['feature']
    nd_content = QN.nodes[index_id]['content']
    nd_reference = QN.nodes[index_id]['reference']
    if nd_feature == 'Central question':
        rgb_map.append({'r': 205, 'g': 37, 'b': 38, 'a': 0})
        color_map.append('red')
        node_size = 20
        title_name = nd_label
    elif nd_feature == 'Question':
        rgb_map.append({'r': 205, 'g': 137, 'b': 38, 'a': 0})
        color_map.append('orange')
        node_size = 15
    elif nd_feature == 'Answer':
        rgb_map.append({'r': 144, 'g': 202, 'b': 249, 'a': 0})
        color_map.append('blue')
    elif nd_feature == 'Follow-up':
        rgb_map.append({'r': 199, 'g': 129, 'b': 32, 'a': 0})
        color_map.append('green')
    node_color = color_map[-1]
    QN_html.add_node(nd_label,size = node_size, color = node_color, feature=nd_feature,content=nd_content,reference=nd_reference)

for index_id in QN.edges():
    start = index_id[0]
    end = index_id[1]
    lk_label = QN.edges[index_id]['label']
    
    lk_from_n = QN.nodes[start]['label']
    lk_to_n = QN.nodes[end]['label']
    
    QN_html.add_edge(lk_from_n,lk_to_n,label=lk_label)

AttributeError: 'function' object has no attribute 'data'

In [6]:
!pip3 show pandas

Name: pandas
Version: 0.20.3
Summary: Powerful data structures for data analysis, time series,and statistics
Home-page: http://pandas.pydata.org
Author: The PyData Development Team
Author-email: pydata@googlegroups.com
License: BSD
Location: /Users/zhiwei/anaconda3/lib/python3.5/site-packages
Requires: python-dateutil, pytz, numpy
Required-by: xarray, seaborn, pymc3, pyLDAvis, odo, ggplot, cufflinks


In [7]:
!pip3 install pandas==0.24.1

    100% |████████████████████████████████| 16.0MB 665kB/s ta 0:00:011   30% |█████████▊                      | 4.9MB 5.7MB/s eta 0:00:02
  Found existing installation: pandas 0.20.3
    Uninstalling pandas-0.20.3:
      Successfully uninstalled pandas-0.20.3
You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# part 3
# visulization

## generate jpg version of the graph
Q = QN
fig = plt.figure(frameon=False)
fig.set_size_inches(35, 25) 
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

pos = nx.kamada_kawai_layout(Q)

nx.draw_networkx(Q,pos,node_color=color_map,arrows=True,arrowstyle='->',arrowsize=25,with_labels=False)
# nx.draw_networkx(Q,pos,node_color=color_map,arrows=True,arrowstyle='->',arrowsize=25,with_labels=False)
# nx.draw_networkx(Q,pos)
node_labels = nx.get_node_attributes(Q,'label')
# nx.draw_networkx_labels(Q,pos,labels=node_labels,font_size = 24)
edge_labels = nx.get_edge_attributes(Q,'label')
# nx.draw_networkx_edge_labels(Q, pos, labels = edge_labels,font_size = 12)
# nx.draw(Q,with_labels = True,font_weight = 'bold',node_size = 1000,node_color='green',node_shape='h')

# for kn,n in enumerate(Q.node):
#     Q.node[n]['viz']={'color':rgb_map[kn]}
# save and illustration
# fig.savefig('QuestionNet8.png', dpi=100)
# plt.show()

title_name = 'test'
# simple = 'QN_simple_' + title_name + '.png'
# fig.savefig(simple, dpi=100)
        
# draw the full graph
nx.draw_networkx_labels(Q,pos,labels=node_labels,font_size = 12)
# nx.draw_networkx_edge_labels(Q, pos, labels = edge_labels,font_size = 9)
        
full = 'QN_full_'+ title_name +'.png'
fig.savefig(full, dpi=100)
plt.show()

## generate the html version of the graph
QN_html.toggle_drag_nodes(True)
# g.toggle_stabilization(True)
QN_html.toggle_hide_edges_on_drag(False)
QN_html.toggle_hide_nodes_on_drag(True)
QN_html.toggle_physics(True)
QN_html.toggle_stabilization(False)
# EdgeOptions.toggle_smoothness(smooth_type = 'curvedCCW')
QN_html.barnes_hut(gravity=-8000,central_gravity=0.3,spring_length=100,spring_strength=0.001,damping=0.09,overlap=0)
# g.force_atlas_2based(gravity=-50,central_gravity=0.01,spring_length=100,spring_strength=0.08,damping=0.4,overlap=0)
# g.hrepulsion(node_distance=120,central_gravity=0.0,spring_length=100,spring_strength=0.01,damping=0.09)
html = 'QN_html_3.html'
QN_html.show(html)

In [4]:
# part 4 
# Question Networks calculation 

# The QN graph generated will be analyzed via following functions:
# 1 _is_answered_. To evaluate whether the node has been answered or not. 
# 2 _describe_. To describe a node in the graph via extracting the 
#   subgraph centralized of the node. 
# 3 _connection_. To describe the connection situation between the two nodes.
# 4 _check_. To check the existence and connectivity of the input node.

# function 1
def _is_answered_(G,NODE):
    # initialization 
    answering_node = []
    answering_id = []
    answering_edge = []
    
    # storing the id of the input node
    node_id = []
    
    # search the whole graph of the node
    for index_id in G.nodes():
        if G.nodes[index_id]['label'] == NODE:
            node_id = index_id
    
    if len(node_id)==0:
        print('The input node does not exist in the graph')
    
    # serach the answering node id
    for index_id in G.edges():
        if index_id[0] == node_id:
            answering_id.append(index_id[1])
    
    if len(answering_id) == 0:
        print('The input node has not been answered')
    else:
        for index1 in range(len(answering_id)):
            index_id = answering_id[index1]
            print('The node has been answered via')
            print(G.nodes[index_id]['label'])

# function 2
def _describe_(G,NODE):
    # initialization
    start_id = []
    end_id = []
    
    # storing the id of the input node
    node_id = []
    
    # search the whole graph of the node
    for index_id in G.nodes():
        if G.nodes[index_id]['label'] == NODE:
            print(index_id)
            node_id = index_id
    
    if len(node_id)==0:
        print('The input node does not exist in the graph')
    
    # search the connection node
    for index_id in G.edges():
        if index_id[0] == node_id:
            end_id.append(index_id[1])
        
        if index_id[1] == node_id:
            start_id.append(index_id[0])
    
    print('end_id')
    print(end_id)
    print('start_id')
    print(start_id)
    
    # build the extracted QN for the input node
    QN_node = Network(height="100%", width="100%", bgcolor="#222222",
                      font_color="white")
    
    QN_node.add_node(NODE,size = 20, color = 'red', feature='selected node',
                     content=G.nodes[node_id]['content'],
                     reference=G.nodes[node_id]['reference'])
    
    for index1 in range(len(start_id)):
        n_id = start_id[index1]
        
        QN_node.add_node(G.nodes[n_id]['label'],
                        size = 15, color = 'yellow',
                        feature = G.nodes[n_id]['feature'],
                        content=G.nodes[n_id]['content'],
                        reference=G.nodes[n_id]['reference'])
        
        edge_label = G.edges[n_id,node_id]['label']
        n_start_label = G.nodes[n_id]['label']
        n_end_label = NODE
        
#         QN_node.add_edge(n_start_label,n_end_label,attr_dict={'label':edge_label})
        QN_node.add_edge(n_start_label,n_end_label,label=edge_label)

    
    for index1 in range(len(end_id)):
        n_id = end_id[index1]
        
        QN_node.add_node(G.nodes[n_id]['label'],
                        size = 15, color = 'green',
                        feature = G.nodes[n_id]['feature'],
                        content=G.nodes[n_id]['content'],
                        reference=G.nodes[n_id]['reference'])
        
        edge_label = G.edges[node_id,n_id]['label']
        n_start_label = NODE
        n_end_label = G.nodes[n_id]['label']
        
#         QN_node.add_edge(n_start_label,n_end_label,attr_dict={'label':edge_label})
        QN_node.add_edge(n_start_label,n_end_label,label=edge_label)

        
    # output the html version of the node graph
    # N3.from_nx(QN)
    QN_node.toggle_drag_nodes(True)
    # g.toggle_stabilization(True)
    QN_node.toggle_hide_edges_on_drag(False)
    QN_node.toggle_hide_nodes_on_drag(True)
    QN_node.toggle_physics(True)
    QN_node.toggle_stabilization(False)
    # EdgeOptions.toggle_smoothness(smooth_type = 'curvedCCW')
    QN_node.barnes_hut(gravity=-10000,central_gravity=0.3,
                       spring_length=200,spring_strength=0.001,
                       damping=0.09,overlap=0)
    # g.force_atlas_2based(gravity=-50,central_gravity=0.01,spring_length=100,spring_strength=0.08,damping=0.4,overlap=0)
    # g.hrepulsion(node_distance=120,central_gravity=0.0,spring_length=100,spring_strength=0.01,damping=0.09)
    html = 'QN_note.html'
    QN_node.show(html)
    
# function 3 
def _connection_(G,NODE1,NODE2):
    
    # initialization
    connection_id = []
    
    # storing the id of the input node
    node_1 = []
    node_2 = []
    
    # search the whole graph of the node
    for index_id in G.nodes():
        if G.nodes[index_id]['label'] == NODE1:
            node_1 = index_id
        
        if G.nodes[index_id]['label'] == NODE2:
            node_2 = index_id
    
    if len(node_1)==0 or len(node_2)==0:
        print('The input node does not exist in the graph')
    
    connection_id = nx.shortest_path(G,node_1,node_2)
    
    print('The shortest route between selected two nodes are:')
#     print(NODE1)
    for index1 in range(len(connection_id)):
        print(G.nodes[connection_id[index1]]['label'])

# function 4
def _check_(G,NODE):
    
    # storing the id of the input node
    node_id = []
    
    # search the whole graph of the node
    for index_id in G.nodes():
        if G.nodes[index_id]['label'] == NODE:
#             print(index_id)
            node_id.append(index_id)
    
    if len(node_id)==0:
        print('The input node does not exist in the graph')
    
    print(node_id)
    print(NODE)
    
    for id1 in range(len(node_id)):
        id_node = node_id[id1]
        _check_edge_(G,id_node)

# sub-fuction of function 4
def _check_edge_(G,node_id): 
    
    print('test id '+node_id)
    
    # initialization
    start_id = []
    end_id = [] 
    
    # search the connection node
    for index_id in G.edges():
        if index_id[0] == node_id:
            end_id.append(index_id[1])
        
        if index_id[1] == node_id:
            start_id.append(index_id[0])
    
    print('end_id')
    print(end_id)
    print('start_id')
    print(start_id)
    
    for index1 in range(len(start_id)):
        n_id = start_id[index1]
        
        print('start from other point')
        print(G.nodes[n_id]['label'])
        print(G.edges[n_id,node_id]['label'])
        
    
    for index1 in range(len(end_id)):
        n_id = end_id[index1]
        
        print('start from other point')
        print(G.nodes[n_id]['label'])
        print(G.edges[node_id,n_id]['label'])

In [5]:
_is_answered_(QN,'numerical methods.')

The node has been answered via
via modelling and simulation
The node has been answered via
melt pool model (melt and solidify within melt pool).
The node has been answered via
laser powder interaction model (powder input).
The node has been answered via
temperautre field and microstructure model (melt and solidify in the whole component).


In [6]:
_describe_(QN,'Laser related parameter')

ad90531
end_id
['ad26690', 'ad31660', 'ad35507']
start_id
['ad26358', 'ad69894', 'ad12631', 'ad481', 'ad4218']


In [7]:
_connection_(QN,'Laser related parameter','Does there exist subtructive manufacturing?')

The shortest route between selected two nodes are:
Laser related parameter
What is the categories of the process parameters?
How to select process parameters of the metal AM?
How to make the building plan of the additive manufacturing?
What is 3D printing (additive manufacturing, AM)?
Why is it additive manufacturing?
Does there exist subtructive manufacturing?


In [8]:
_check_(QN,'Laser related parameter')

['ad90531']
Laser related parameter
test id ad90531
end_id
['ad26690', 'ad31660', 'ad35507']
start_id
['ad26358', 'ad69894', 'ad12631', 'ad481', 'ad4218']
start from other point
Laser power.
specification
start from other point
melt pool model (melt and solidify within melt pool).
answer
start from other point
Laser inclined angle.
specification
start from other point
What is the categories of the process parameters?
answer
start from other point
temperautre field and microstructure model (melt and solidify in the whole component).
answer
start from other point
Laser scanning pattern.
specification
start from other point
Laser scannning speed.
specification
start from other point
laser powder interaction model (powder input).
answer
